In [4]:
%run ./load_src_module.py

In [23]:
import os

from tqdm import tqdm
import tensorflow as tf

from src.datasets import PersonalityCaptions, DatasetManager
from src.image_encoders import InceptionResNetEncoder
from src.transformer import TransformerGenerator
from src.utils import MultiCheckpointManager
from src.evaluate import _run_beam_search

In [8]:
pc = PersonalityCaptions("/Users/akshaykurmi/NEU/Image-Captioning/stylized-captions/data/personality_captions_data")
dm = DatasetManager(pc, 20)

In [24]:
def list_test_captions(dataset_manager, checkpoint_number):
    generator = TransformerGenerator(token_vocab_size=dataset_manager.tokenizer.vocab_size,
                                     style_vocab_size=dataset_manager.style_encoder.num_classes,
                                     model_dim=512, style_dim=64, pffn_dim=2048, z_dim=512,
                                     encoder_blocks=2, decoder_blocks=6, num_attention_heads=8, max_pe=64,
                                     dropout=0.1, stylize=True)
    checkpoint_manager = MultiCheckpointManager("/Users/akshaykurmi/NEU/Image-Captioning/stylized-captions/results/run_1/checkpoints", {
        "generator": {"generator": generator}
    })
    checkpoint_manager.restore({"generator": checkpoint_number})

    num_test_samples = 10000
    batch_size = 32
    test_dataset = dataset_manager.load_generator_dataset("test", batch_size, 1)

    predictions = {}
    sample_id = 0
    for batch in tqdm(test_dataset, desc="Batch", unit="batch", total=int(num_test_samples / batch_size) + 1):
        encoder_output, caption, style, additional_captions = batch
        sequences, sequences_logits = _run_beam_search(generator, encoder_output, style, 5, 20,
                                                       dataset_manager.tokenizer.start_id,
                                                       dataset_manager.tokenizer.end_id)
        for s, c, acs in zip(sequences.numpy(), caption.numpy(), additional_captions.numpy()):
            predictions[sample_id] = _seq_to_text(dataset_manager, s[0])
            sample_id += 1

    with open("preds.json", "w") as f:
        json.dump(predictions, f)

In [ ]:
list_test_captions(dm, 231)

Batch:   0%|          | 0/313 [00:00<?, ?batch/s]